# Практическая работа №7

По теме: "Ансамблевое обучение"

Выполнил: студент группы ИВБО-07-21 Чан Минь Ханг

Принял: старший преподаватель кафедры ПМ Юрченков И.А.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn import tree, ensemble
import catboost as cb

from sklearn.metrics import f1_score, r2_score

import time
import warnings
warnings.filterwarnings(action='ignore')

## 1. Загрузка данных
Найти данные для задачи классификации или для задачи регрессии. 

**Набор данных для прогнозирования заболеваний сердца** содержит 1888 записей, объединенных из пяти общедоступных наборов данных о сердечно-сосудистых заболеваниях. Он включает в себя 14 функций, которые имеют решающее значение для прогнозирования рисков сердечного приступа и инсульта, охватывая как медицинские, так и демографические факторы. Ниже приведено подробное описание каждой функции:

- age: возраст пациента (числовой).
- sex: Пол пациента. Значения: 1 = мужчина, 0 = женщина.
- cp: Тип боли в груди. Значения: 0 = типичная стенокардия, 1 = атипичная стенокардия, 2 = неангинальная боль, 3 = бессимптомное течение.
- trestbps: кровяное давление в состоянии покоя (в мм рт. ст.) (числовое).
- chol: уровень холестерина в сыворотке (в мг/дл) (числовой).
- fbs: уровень сахара в крови натощак > 120 мг/дл. Значения: 1 = истина, 0 = ложь.
- restecg: результаты электрокардиографии в покое. Значения: 0 = норма, 1 = аномалия ST-T, 2 = гипертрофия левого желудочка.
- thalach: достигнутая максимальная частота пульса (числовая).
- exang: стенокардия, вызванная физической нагрузкой. Значения: 1 = да, 0 = нет.
- oldpeak: депрессия ST, вызванная нагрузкой, по сравнению с отдыхом (числовое значение).
- slope: наклон пикового сегмента ST при нагрузке. Значения: 0 = восходящий, 1 = ровный, 2 = нисходящий.
- ca: количество крупных сосудов (0–3), окрашенных при рентгеноскопии. Значения: 0, 1, 2, 3.
- thal: Типы талассемии. Значения: 1 = Нормальный, 2 = Исправленный дефект, 3 = Обратимый дефект.
- target: переменная результата (риск сердечного приступа). Значения: 1 = больше вероятность сердечного приступа, 0 = меньше вероятность сердечного приступа.

In [2]:
url = "https://www.kaggle.com/datasets/mfarhaannazirkhan/heart-dataset/data?select=cleaned_merged_heart_dataset.csv"

data = pd.read_csv('heart_dataset.csv')

data

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal,target
421,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
72,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
367,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
363,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
971,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,77,1,0,125,304,0,0,162,1,0.0,2,3,2,0
463,77,1,0,125,304,0,0,162,1,0.0,2,3,2,0
690,77,1,0,125,304,0,0,162,1,0.0,2,3,2,0
1837,77,0,0,125,304,0,0,162,1,0.0,2,3,2,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1888 entries, 421 to 1528
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1888 non-null   int64  
 1   sex       1888 non-null   int64  
 2   cp        1888 non-null   int64  
 3   trestbps  1888 non-null   int64  
 4   chol      1888 non-null   int64  
 5   fbs       1888 non-null   int64  
 6   restecg   1888 non-null   int64  
 7   thalachh  1888 non-null   int64  
 8   exang     1888 non-null   int64  
 9   oldpeak   1888 non-null   float64
 10  slope     1888 non-null   int64  
 11  ca        1888 non-null   int64  
 12  thal      1888 non-null   int64  
 13  target    1888 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 221.2 KB


In [4]:
predicts = data.drop('target', axis=1)
target = data['target']

In [5]:
A_train, A_test, y_train, y_test = train_test_split(predicts, target, train_size=0.8)

print(A_train.shape)
print(A_test.shape)

(1510, 13)
(378, 13)


## 2. Bagging (Случайный лес)
Реализовать баггинг. 

In [6]:
random_forest = ensemble.RandomForestClassifier()

params_grid = {
    'max_depth': [3, 20],
    'min_samples_leaf': [3, 10],
    'min_samples_split': [6, 12]
}

search = GridSearchCV(estimator=random_forest, param_grid=params_grid, scoring='r2', cv=4)
search.fit(A_train, y_train)
model_bagging = search.best_estimator_

start_time = time.time()
model_bagging.fit(A_train, y_train)
training_time = time.time() - start_time

In [10]:
y_preds_d = model_bagging.predict(A_train)
print('R2 мера для тренировочных данных: ', r2_score(y_preds_d, y_train))
print('F1 мера для тренировочных данных:', f1_score(y_train, y_preds_d, average='macro'))

y_preds = model_bagging.predict(A_test)
print('R2 мера для тестовых данных: ', r2_score(y_preds, y_test))
print('F1 мера для тестовых данных: ', f1_score(y_test, y_preds, average='macro'))

print(f'Время обучения: {training_time:.5f} сек')

R2 мера для тренировочных данных:  0.9761438321536906
F1 мера для тренировочных данных: 0.9940353376617733
R2 мера для тестовых данных:  0.9148073022312373
F1 мера для тестовых данных:  0.9787353735373537
Время обучения: 0.26300 сек


## 3. Boosting (CatBoost)
Реализовать бустинг на тех же данных, что использовались для баггинга. 

In [11]:
model_catboost = cb.CatBoostClassifier(iterations=3000, task_type='GPU', devices='0')

start_time = time.time()
model_catboost.fit(A_train, y_train)
training_time = time.time() - start_time

Learning rate set to 0.012315
0:	learn: 0.6821460	total: 48.3ms	remaining: 2m 24s
1:	learn: 0.6692936	total: 96.5ms	remaining: 2m 24s
2:	learn: 0.6577053	total: 142ms	remaining: 2m 21s
3:	learn: 0.6461471	total: 189ms	remaining: 2m 21s
4:	learn: 0.6348077	total: 233ms	remaining: 2m 19s
5:	learn: 0.6245705	total: 272ms	remaining: 2m 15s
6:	learn: 0.6145342	total: 318ms	remaining: 2m 15s
7:	learn: 0.6056174	total: 363ms	remaining: 2m 15s
8:	learn: 0.5955555	total: 401ms	remaining: 2m 13s
9:	learn: 0.5861418	total: 446ms	remaining: 2m 13s
10:	learn: 0.5770952	total: 491ms	remaining: 2m 13s
11:	learn: 0.5688714	total: 539ms	remaining: 2m 14s
12:	learn: 0.5597954	total: 585ms	remaining: 2m 14s
13:	learn: 0.5507813	total: 625ms	remaining: 2m 13s
14:	learn: 0.5421152	total: 666ms	remaining: 2m 12s
15:	learn: 0.5341934	total: 707ms	remaining: 2m 11s
16:	learn: 0.5278941	total: 748ms	remaining: 2m 11s
17:	learn: 0.5198569	total: 795ms	remaining: 2m 11s
18:	learn: 0.5149285	total: 834ms	remainin

157:	learn: 0.2324908	total: 8.05s	remaining: 2m 24s
158:	learn: 0.2318638	total: 8.1s	remaining: 2m 24s
159:	learn: 0.2313988	total: 8.14s	remaining: 2m 24s
160:	learn: 0.2309760	total: 8.2s	remaining: 2m 24s
161:	learn: 0.2306316	total: 8.25s	remaining: 2m 24s
162:	learn: 0.2300223	total: 8.3s	remaining: 2m 24s
163:	learn: 0.2293021	total: 8.35s	remaining: 2m 24s
164:	learn: 0.2287053	total: 8.4s	remaining: 2m 24s
165:	learn: 0.2282087	total: 8.44s	remaining: 2m 24s
166:	learn: 0.2278735	total: 8.49s	remaining: 2m 24s
167:	learn: 0.2274352	total: 8.54s	remaining: 2m 23s
168:	learn: 0.2269494	total: 8.59s	remaining: 2m 23s
169:	learn: 0.2256557	total: 8.64s	remaining: 2m 23s
170:	learn: 0.2248867	total: 8.69s	remaining: 2m 23s
171:	learn: 0.2240491	total: 8.74s	remaining: 2m 23s
172:	learn: 0.2230736	total: 8.8s	remaining: 2m 23s
173:	learn: 0.2224602	total: 8.85s	remaining: 2m 23s
174:	learn: 0.2218665	total: 8.89s	remaining: 2m 23s
175:	learn: 0.2211361	total: 8.95s	remaining: 2m 23

315:	learn: 0.1518207	total: 16.3s	remaining: 2m 18s
316:	learn: 0.1515266	total: 16.3s	remaining: 2m 18s
317:	learn: 0.1513838	total: 16.4s	remaining: 2m 18s
318:	learn: 0.1510876	total: 16.4s	remaining: 2m 18s
319:	learn: 0.1503819	total: 16.5s	remaining: 2m 18s
320:	learn: 0.1498342	total: 16.6s	remaining: 2m 18s
321:	learn: 0.1495722	total: 16.6s	remaining: 2m 18s
322:	learn: 0.1492949	total: 16.7s	remaining: 2m 18s
323:	learn: 0.1491860	total: 16.7s	remaining: 2m 18s
324:	learn: 0.1488857	total: 16.8s	remaining: 2m 18s
325:	learn: 0.1486295	total: 16.8s	remaining: 2m 18s
326:	learn: 0.1481911	total: 16.9s	remaining: 2m 18s
327:	learn: 0.1476917	total: 16.9s	remaining: 2m 18s
328:	learn: 0.1474417	total: 17s	remaining: 2m 18s
329:	learn: 0.1471707	total: 17.1s	remaining: 2m 18s
330:	learn: 0.1467903	total: 17.1s	remaining: 2m 18s
331:	learn: 0.1462104	total: 17.2s	remaining: 2m 18s
332:	learn: 0.1454642	total: 17.2s	remaining: 2m 18s
333:	learn: 0.1450516	total: 17.3s	remaining: 2m

472:	learn: 0.1094808	total: 24.8s	remaining: 2m 12s
473:	learn: 0.1092350	total: 24.8s	remaining: 2m 12s
474:	learn: 0.1089377	total: 24.9s	remaining: 2m 12s
475:	learn: 0.1085750	total: 24.9s	remaining: 2m 12s
476:	learn: 0.1083259	total: 25s	remaining: 2m 12s
477:	learn: 0.1080130	total: 25s	remaining: 2m 11s
478:	learn: 0.1078875	total: 25.1s	remaining: 2m 11s
479:	learn: 0.1076310	total: 25.1s	remaining: 2m 11s
480:	learn: 0.1073426	total: 25.1s	remaining: 2m 11s
481:	learn: 0.1070667	total: 25.2s	remaining: 2m 11s
482:	learn: 0.1066678	total: 25.3s	remaining: 2m 11s
483:	learn: 0.1064207	total: 25.3s	remaining: 2m 11s
484:	learn: 0.1059911	total: 25.4s	remaining: 2m 11s
485:	learn: 0.1058102	total: 25.4s	remaining: 2m 11s
486:	learn: 0.1055679	total: 25.5s	remaining: 2m 11s
487:	learn: 0.1051899	total: 25.5s	remaining: 2m 11s
488:	learn: 0.1050554	total: 25.6s	remaining: 2m 11s
489:	learn: 0.1046665	total: 25.6s	remaining: 2m 11s
490:	learn: 0.1046143	total: 25.7s	remaining: 2m 1

634:	learn: 0.0809619	total: 33s	remaining: 2m 3s
635:	learn: 0.0808211	total: 33.1s	remaining: 2m 2s
636:	learn: 0.0806545	total: 33.1s	remaining: 2m 2s
637:	learn: 0.0804359	total: 33.2s	remaining: 2m 2s
638:	learn: 0.0802528	total: 33.2s	remaining: 2m 2s
639:	learn: 0.0801579	total: 33.3s	remaining: 2m 2s
640:	learn: 0.0800077	total: 33.3s	remaining: 2m 2s
641:	learn: 0.0798887	total: 33.4s	remaining: 2m 2s
642:	learn: 0.0797073	total: 33.4s	remaining: 2m 2s
643:	learn: 0.0795546	total: 33.5s	remaining: 2m 2s
644:	learn: 0.0795409	total: 33.5s	remaining: 2m 2s
645:	learn: 0.0793966	total: 33.6s	remaining: 2m 2s
646:	learn: 0.0791312	total: 33.6s	remaining: 2m 2s
647:	learn: 0.0790773	total: 33.7s	remaining: 2m 2s
648:	learn: 0.0790251	total: 33.7s	remaining: 2m 2s
649:	learn: 0.0789818	total: 33.8s	remaining: 2m 2s
650:	learn: 0.0788813	total: 33.8s	remaining: 2m 2s
651:	learn: 0.0787946	total: 33.9s	remaining: 2m 2s
652:	learn: 0.0786972	total: 33.9s	remaining: 2m 1s
653:	learn: 0.

...

2332:	learn: 0.0180269	total: 2m 14s	remaining: 38.5s
2333:	learn: 0.0180170	total: 2m 14s	remaining: 38.5s
2334:	learn: 0.0179938	total: 2m 14s	remaining: 38.4s
2335:	learn: 0.0179919	total: 2m 15s	remaining: 38.4s
2336:	learn: 0.0179909	total: 2m 15s	remaining: 38.3s
2337:	learn: 0.0179874	total: 2m 15s	remaining: 38.3s
2338:	learn: 0.0179847	total: 2m 15s	remaining: 38.2s
2339:	learn: 0.0179835	total: 2m 15s	remaining: 38.1s
2340:	learn: 0.0179605	total: 2m 15s	remaining: 38.1s
2341:	learn: 0.0179482	total: 2m 15s	remaining: 38s
2342:	learn: 0.0179335	total: 2m 15s	remaining: 38s
2343:	learn: 0.0179200	total: 2m 15s	remaining: 37.9s
2344:	learn: 0.0179166	total: 2m 15s	remaining: 37.8s
2345:	learn: 0.0178989	total: 2m 15s	remaining: 37.8s
2346:	learn: 0.0178715	total: 2m 15s	remaining: 37.7s
2347:	learn: 0.0178606	total: 2m 15s	remaining: 37.7s
2348:	learn: 0.0178596	total: 2m 15s	remaining: 37.6s
2349:	learn: 0.0178195	total: 2m 15s	remaining: 37.6s
2350:	learn: 0.0177759	total: 2m

2487:	learn: 0.0166904	total: 2m 23s	remaining: 29.6s
2488:	learn: 0.0166893	total: 2m 24s	remaining: 29.6s
2489:	learn: 0.0166890	total: 2m 24s	remaining: 29.5s
2490:	learn: 0.0166816	total: 2m 24s	remaining: 29.5s
2491:	learn: 0.0166640	total: 2m 24s	remaining: 29.4s
2492:	learn: 0.0166517	total: 2m 24s	remaining: 29.3s
2493:	learn: 0.0166352	total: 2m 24s	remaining: 29.3s
2494:	learn: 0.0166081	total: 2m 24s	remaining: 29.2s
2495:	learn: 0.0166068	total: 2m 24s	remaining: 29.2s
2496:	learn: 0.0166048	total: 2m 24s	remaining: 29.1s
2497:	learn: 0.0166043	total: 2m 24s	remaining: 29.1s
2498:	learn: 0.0166033	total: 2m 24s	remaining: 29s
2499:	learn: 0.0166023	total: 2m 24s	remaining: 28.9s
2500:	learn: 0.0166000	total: 2m 24s	remaining: 28.9s
2501:	learn: 0.0165803	total: 2m 24s	remaining: 28.8s
2502:	learn: 0.0165797	total: 2m 24s	remaining: 28.8s
2503:	learn: 0.0165786	total: 2m 24s	remaining: 28.7s
2504:	learn: 0.0165762	total: 2m 24s	remaining: 28.6s
2505:	learn: 0.0165512	total: 

2642:	learn: 0.0154629	total: 2m 32s	remaining: 20.6s
2643:	learn: 0.0154556	total: 2m 32s	remaining: 20.6s
2644:	learn: 0.0154416	total: 2m 32s	remaining: 20.5s
2645:	learn: 0.0154414	total: 2m 32s	remaining: 20.4s
2646:	learn: 0.0154277	total: 2m 32s	remaining: 20.4s
2647:	learn: 0.0154248	total: 2m 32s	remaining: 20.3s
2648:	learn: 0.0154130	total: 2m 32s	remaining: 20.3s
2649:	learn: 0.0154103	total: 2m 33s	remaining: 20.2s
2650:	learn: 0.0154093	total: 2m 33s	remaining: 20.2s
2651:	learn: 0.0154086	total: 2m 33s	remaining: 20.1s
2652:	learn: 0.0154060	total: 2m 33s	remaining: 20s
2653:	learn: 0.0154058	total: 2m 33s	remaining: 20s
2654:	learn: 0.0154047	total: 2m 33s	remaining: 19.9s
2655:	learn: 0.0154035	total: 2m 33s	remaining: 19.9s
2656:	learn: 0.0153981	total: 2m 33s	remaining: 19.8s
2657:	learn: 0.0153980	total: 2m 33s	remaining: 19.8s
2658:	learn: 0.0153957	total: 2m 33s	remaining: 19.7s
2659:	learn: 0.0153858	total: 2m 33s	remaining: 19.6s
2660:	learn: 0.0153724	total: 2m

2799:	learn: 0.0144953	total: 2m 41s	remaining: 11.6s
2800:	learn: 0.0144871	total: 2m 41s	remaining: 11.5s
2801:	learn: 0.0144868	total: 2m 41s	remaining: 11.4s
2802:	learn: 0.0144707	total: 2m 41s	remaining: 11.4s
2803:	learn: 0.0144670	total: 2m 41s	remaining: 11.3s
2804:	learn: 0.0144668	total: 2m 42s	remaining: 11.3s
2805:	learn: 0.0144524	total: 2m 42s	remaining: 11.2s
2806:	learn: 0.0144517	total: 2m 42s	remaining: 11.1s
2807:	learn: 0.0144447	total: 2m 42s	remaining: 11.1s
2808:	learn: 0.0144444	total: 2m 42s	remaining: 11s
2809:	learn: 0.0144351	total: 2m 42s	remaining: 11s
2810:	learn: 0.0144328	total: 2m 42s	remaining: 10.9s
2811:	learn: 0.0144286	total: 2m 42s	remaining: 10.9s
2812:	learn: 0.0144202	total: 2m 42s	remaining: 10.8s
2813:	learn: 0.0144096	total: 2m 42s	remaining: 10.7s
2814:	learn: 0.0144016	total: 2m 42s	remaining: 10.7s
2815:	learn: 0.0143974	total: 2m 42s	remaining: 10.6s
2816:	learn: 0.0143964	total: 2m 42s	remaining: 10.6s
2817:	learn: 0.0143948	total: 2m

2952:	learn: 0.0135938	total: 2m 50s	remaining: 2.71s
2953:	learn: 0.0135779	total: 2m 50s	remaining: 2.65s
2954:	learn: 0.0135772	total: 2m 50s	remaining: 2.59s
2955:	learn: 0.0135763	total: 2m 50s	remaining: 2.54s
2956:	learn: 0.0135668	total: 2m 50s	remaining: 2.48s
2957:	learn: 0.0135585	total: 2m 50s	remaining: 2.42s
2958:	learn: 0.0135489	total: 2m 50s	remaining: 2.36s
2959:	learn: 0.0135417	total: 2m 50s	remaining: 2.31s
2960:	learn: 0.0135373	total: 2m 50s	remaining: 2.25s
2961:	learn: 0.0135371	total: 2m 50s	remaining: 2.19s
2962:	learn: 0.0135327	total: 2m 50s	remaining: 2.13s
2963:	learn: 0.0135234	total: 2m 50s	remaining: 2.08s
2964:	learn: 0.0135220	total: 2m 51s	remaining: 2.02s
2965:	learn: 0.0135186	total: 2m 51s	remaining: 1.96s
2966:	learn: 0.0134939	total: 2m 51s	remaining: 1.9s
2967:	learn: 0.0134937	total: 2m 51s	remaining: 1.84s
2968:	learn: 0.0134935	total: 2m 51s	remaining: 1.79s
2969:	learn: 0.0134741	total: 2m 51s	remaining: 1.73s
2970:	learn: 0.0134720	total:

In [12]:
y_preds_t = model_catboost.predict(A_train, task_type='CPU')
print('R2 мера для тренировочных данных', r2_score(y_preds_t, y_train))
print('F1 мера для тренировочных данных:', f1_score(y_train, y_preds_t, average='macro'))

y_preds = model_catboost.predict(A_test, task_type='CPU')
print('R2 мера для тестовых данных', r2_score(y_preds, y_test))
print('F1 мера для тестовых данных: ', f1_score(y_test, y_preds, average='macro'))

print(f'Время обучения: {training_time:.5f} сек')

R2 мера для тренировочных данных 1.0
F1 мера для тренировочных данных: 1.0
R2 мера для тестовых данных 0.8727272727272727
F1 мера для тестовых данных:  0.968164846579265
Время обучения: 175.49496 сек


## 4. Сравнение результатов
Сравнить результаты работы алгоритмов (время работы и качество 
моделей). Сделать выводы. 

*Бустинг дольше выполняется, и имеет менее хорошое качество, чем баггинг.*

## 5. Отчет
Оформить отчет о проделанной работе. 